# 📐 Chapter 11: Resolution & Scaling (Size Matters)

In the glory days of SNES, games were 256x224 pixels. Today, players have 4K monitors. 

If you just make a window `800x600`, it will look tiny on a modern screen. But if you code it for `1920x1080`, it won't fit on a laptop! 🤯

The Solution: **Virtual Resolution**.

## 1. The Strategy
1.  **Render Small**: Draw your game to a small surface (e.g., 320x180 for retro pixel art, or 1280x720 for HD).
2.  **Scale Up**: Stretch that surface to fill the user's actual window.

In [ ]:
import pygame

# 1. Define your logical internal resolution
GAME_W = 400
GAME_H = 300

# 2. Create the display (Window size the user sees)
WINDOW_W = 800
WINDOW_H = 600
screen = pygame.display.set_mode((WINDOW_W, WINDOW_H), pygame.RESIZABLE)

# 3. Create the 'Canvas' (Where we actually draw)
canvas = pygame.Surface((GAME_W, GAME_H))

# Game Loop
running = True
while running:
    # ... Event handling ...
    
    # DRAWING STEP
    # IMPORTANT: Draw everything to 'canvas', NOT 'screen'
    canvas.fill((20, 20, 20))
    pygame.draw.circle(canvas, (255, 0, 0), (GAME_W//2, GAME_H//2), 50)
    
    # SCALING STEP
    # Scale the canvas to fit the window
    scaled_canvas = pygame.transform.scale(canvas, (WINDOW_W, WINDOW_H))
    
    # Blit the big scaled image onto the real screen
    screen.blit(scaled_canvas, (0, 0))
    pygame.display.flip()

## 2. Handling Resizing 🔄

What if the user grabs the corner of the window and drags it? Pygame sends a `VIDEORESIZE` event!

We need to catch this event and update our `WINDOW_W` and `WINDOW_H` variables.

In [ ]:
# Inside the Event Loop:
for event in pygame.event.get():
    if event.type == pygame.VIDEORESIZE:
        WINDOW_W, WINDOW_H = event.w, event.h
        # Re-create the display surface if needed
        screen = pygame.display.set_mode((WINDOW_W, WINDOW_H), pygame.RESIZABLE)

## 3. Maintaining Aspect Ratio (Black Bars) 🎬

If you just stretch `400x300` to `1000x300`, your circles will turn into ovals. Gross. 🤮

To fix this, we calculate the "Best Fit" scale and center the image, leaving black bars on the sides (Letterboxing).

In [ ]:
def draw_game_window():
    # Calculate scale factor
    scale_x = WINDOW_W / GAME_W
    scale_y = WINDOW_H / GAME_H
    scale = min(scale_x, scale_y) # Take the smaller scale to fit inside
    
    new_w = int(GAME_W * scale)
    new_h = int(GAME_H * scale)
    
    # Scale only to the kept aspect ratio
    scaled_canvas = pygame.transform.scale(canvas, (new_w, new_h))
    
    # Calculate centering offset
    offset_x = (WINDOW_W - new_w) // 2
    offset_y = (WINDOW_H - new_h) // 2
    
    # Clear screen (Black bars)
    screen.fill((0, 0, 0))
    # Draw centered game
    screen.blit(scaled_canvas, (offset_x, offset_y))

## 🛠️ Challenge: The Universal Screen

1.  Take your game code.
2.  Wrap all drawing calls to use a `canvas` surface (640x360). 360p is exactly 1/3rd of 1080p!
3.  Implement the Letterboxing logic above.
4.  Run the game, maximize the window, and marvel at your crisp pixels! ✨